Your job is to help implement the ALM and EXAM models into python code, based on a textual description of the models, and preexisting R code. ALM (Associative Learning Model)
The basis of the model is an associative neural network that uses a delta learning rule to associate inputs to outputs. Responses to stimuli within the training domain are obtained by simple activation of the network. The underlying associative learning model is a simple two-layer connectionist network that updates weight strengths using the delta learning rule. The $M$ nodes along the input of the model represent the possible values along some input domain. The $N$ output nodes represent the possible responses along the output range. A real stimulus/response will be mapped to the nodes via some psychophysical function $\psi(x)$. The equations presented will ignore this transformation to simplify things. When an input stimulus $X$ is presented, it will cause a Gaussian activation of input nodes according to the function $a_i(X)=e^{-\gamma \cdot\left(X-X_i\right)^2}$, where $\gamma$ is a scaling parameter describing the steepness of the gradient. The input activation is then normalized so that the area under the curve is equal to 1 by dividing all values by the sum of all input activation values. The activation of each output node is calculated by summing the products of the input nodes and the weights that connect them to a particular output node. This is shown in the formula $O_j(X)=\sum_{i=1}^M w_{j i} \cdot a_i(X)$, where $w_{j i}$ designates the strength of association between input node $X_i$ and output node $Y_j$. At this point we have a distribution that should describe the responses in a stochastic simulation. In order to obtain deterministic predictions, the expected value of the output activation is calculated. First the probability of choosing a particular output given a specific input is calculated according to $P\left[Y_j \mid X\right]=\frac{O_j(X)}{\sum_{k=1}^{\mathrm{L}} O_k(X)}$. The mean output given stimulus $X$ is the weighted average $m(X)=\sum_{j=1}^L Y_j \cdot P\left[Y_j \mid X\right]$.
In addition to creating associations, the model allows for learning. This step adjusts weight connections to improve accuracy. The first step is obtaining an error signal. When participants perform the task, they are shown a feedback signal $Z$ that demonstrates what the proper response should have been. This signal activates the output nodes using another Gaussian similarity function $f_j(Z)=e^{-\gamma \cdot\left(Z-Y_j\right)^2}$. This feedback signal is used to update the weights according to the delta learning rule:
$w_{j i}(t+1)=w_{j i}(t)+\alpha \cdot\left\{f_j[Z(t)]-O_j[X(t)]\right\} \cdot a_i[X(t)]$
```{r fig.width=12,fig.height=10}
input.activation<-function(x.target, c){return(exp((-1*c)*(x.target-inputNodes)^2))}
output.activation<-function(x.target, weights, c){return(weights%*%input.activation(x.target, c))}
mean.prediction<-function(x.target, weights, c){
  probability<-output.activation(x.target, weights, c)/sum(output.activation(x.target, weights, c))
  return(outputNodes%*%probability) # integer prediction
}
update.weights<-function(x.new, y.new, weights, c, lr){
  y.feedback.activation<-exp(-1*c*(y.new-outputNodes)^2)
  x.feedback.activation<-output.activation(x.new, weights, c)
  return(weights+lr*(y.feedback.activation-x.feedback.activation)%*%t(input.activation(x.new, c)))
}

In [15]:
# import sys
# !{sys.executable} -m pip install numpy

import numpy as np

class ALM:
    def __init__(self, input_nodes, output_nodes, gamma):
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.gamma = gamma
        #self.weights = np.zeros((len(output_nodes), len(input_nodes)))
        self.weights = np.random.randn(len(output_nodes), len(input_nodes))


    def input_activation(self, x_target):
        return np.exp(-self.gamma * (x_target - self.input_nodes) ** 2)

    def output_activation(self, x_target):
        return self.weights @ self.input_activation(x_target)

    def mean_prediction(self, x_target):
        probability = self.output_activation(x_target) / np.sum(self.output_activation(x_target))
        return self.output_nodes @ probability

    def update_weights(self, x_new, y_new, learning_rate):
        y_feedback_activation = np.exp(-self.gamma * (y_new - self.output_nodes) ** 2)
        x_feedback_activation = self.output_activation(x_new)
        self.weights += learning_rate * np.outer((y_feedback_activation - x_feedback_activation), self.input_activation(x_new))
    
    def generate_data(self, x, function_type="linear", noise=None):
        if function_type == "linear":
            y = np.round(2.2 * x + 30, 0)
        elif function_type == "exponential":
            y = np.round(200 * (1 - np.exp(-x / 25)), 0)
        elif function_type == "quadratic":
            y = np.round(210 - ((x - 50) ** 2) / 12, 0)
        else:
            raise ValueError("function_type must be linear, exponential, or quadratic")

        if noise is not None:
            y += np.round(np.random.normal(0, noise, len(y)), 2)

        return x, y

In [3]:
input_nodes = np.linspace(0, 1, 10)
output_nodes = np.linspace(0, 1, 10)
gamma = 1

alm = ALM(input_nodes, output_nodes, gamma)

x_target = 0.5
print("Input activation:", alm.input_activation(x_target))
print("Output activation:", alm.output_activation(x_target))
print("Mean prediction:", alm.mean_prediction(x_target))

x_new = 0.6
y_new = 0.7
learning_rate = 0.1
alm.update_weights(x_new, y_new, learning_rate)
print("Updated weights:", alm.weights)

Input activation: [0.77880078 0.85964603 0.92574127 0.97260448 0.99691834 0.99691834
 0.97260448 0.92574127 0.85964603 0.77880078]
Output activation: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Mean prediction: nan
Updated weights: [[0.04274149 0.04823852 0.05311472 0.05705747 0.05979803 0.06114175
  0.06099097 0.05935671 0.05635739 0.05220458]
 [0.04932249 0.0556659  0.0612929  0.06584273 0.06900525 0.07055586
  0.07038186 0.06849598 0.06503485 0.06024262]
 [0.05552862 0.06267021 0.06900525 0.07412757 0.07768802 0.07943374
  0.07923785 0.07711468 0.07321804 0.06782281]
 [0.06099097 0.06883507 0.07579328 0.08141949 0.08533018 0.08724763
  0.08703247 0.08470044 0.08042049 0.07449453]
 [0.0653568  0.07376241 0.0812187  0.08724763 0.09143826 0.09349297
  0.09326241 0.09076344 0.08617712 0.07982698]
 [0.06832707 0.07711468 0.08490984 0.09121277 0.09559385 0.09774193
  0.09750089 0.09488836 0.09009361 0.08345487]
 [0.06969016 0.07865308 0.08660374 0.09303241 0.09750089 0.09969183
  0.09944598 0.0967813

/var/folders/fq/33ygfd7x1b19zvrs2p3d_w940000gr/T/ipykernel_4865/3603499747.py:20: RuntimeWarning: invalid value encountered in divide
  probability = self.output_activation(x_target) / np.sum(self.output_activation(x_target))


In [16]:
env_types = ["linear", "exponential", "quadratic"]

low_density_train_block = np.array([30.5, 36.0, 41.0, 46.5, 53.5, 59.0, 64.0, 69.5])
med_density_train_block = np.array([
    30.0, 31.5, 33.0, 34.5, 36.5, 38.5, 41.0, 43.5, 46.0,
    48.5, 51.5, 54.0, 56.5, 59.0, 61.5, 63.5, 65.5, 67.0, 68.5, 70.0
])

low_density_blocks = 25
med_density_blocks = 10


input_nodes = np.linspace(0, 100, 100)
output_nodes = np.linspace(0, 100, 100)
gamma = .01
learning_rate = 0.1

alm = ALM(input_nodes, output_nodes, gamma)


for env_type in env_types:
    print(f"Environment type: {env_type}")

    # Low density training
    print("Low density training:")
    for _ in range(low_density_blocks):
        x, y = alm.generate_data(low_density_train_block, function_type=env_type)
        for xi, yi in zip(x, y):
            alm.update_weights(xi, yi, learning_rate)
        print("Mean predictions for low density training:", [alm.mean_prediction(xi) for xi in low_density_train_block])

    # Reset weights for the next simulation
    alm.weights = np.random.randn(len(output_nodes), len(input_nodes))


    # Medium density training
    print("Medium density training:")
    for _ in range(med_density_blocks):
        x, y = alm.generate_data(med_density_train_block, function_type=env_type)
        for xi, yi in zip(x, y):
            alm.update_weights(xi, yi, learning_rate)
        print("Mean predictions for medium density training:", [alm.mean_prediction(xi) for xi in med_density_train_block])

    # Reset weights for the next simulation
    alm.weights = np.random.randn(len(output_nodes), len(input_nodes))


   

Environment type: linear
Low density training:
Mean predictions for low density training: [83.88629455904419, 166.69601913471706, -197.26723519193166, 68.96428473839808, 88.52807512053397, 63.313709130170466, 32.30919747128341, 20.92296823404772]
Mean predictions for low density training: [41.3867822899429, 84.20909305440864, 96.43598622308163, 65.47908985214525, -34.8701839422575, 91.53559236570965, -6.269724228783982, -24.80508294020117]
Mean predictions for low density training: [138.80942479419778, -2.5719047729238156, -220.45501086582087, 99.38353551686973, -33.912823152724826, 174.6429274680901, -43.07094053633337, -71.84733328996847]
Mean predictions for low density training: [-146.99368916757697, -238.80853133652954, -66.77534843486812, 284.7448240207383, -41.95487507537001, 330.19128963194686, -76.04608715454349, -118.23706322551052]
Mean predictions for low density training: [15.35389627328551, 76.23605912230971, -49.58129432848504, -413.15160437494444, -45.01687578565631, 53